<a href="https://colab.research.google.com/github/jh-4166640/Machine-Learning-Lecture/blob/main/Yolov5BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [20]:
# Download latest version
path = kagglehub.dataset_download("prasunroy/natural-images")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'natural-images' dataset.
Path to dataset files: /kaggle/input/natural-images


In [36]:
target_base = "/content/data"

# 🔥 이전 데이터 삭제 (다시 만들기)
if os.path.exists(target_base):
    shutil.rmtree(target_base)
    print("기존 데이터 삭제 완료!")

기존 데이터 삭제 완료!


In [37]:
import os
import shutil
import random

# 기존 폴더 경로
src_base = "/kaggle/input/natural-images/natural_images"


# 새 YOLO 데이터셋 폴더
target_base = "/content/data"
os.makedirs(f"{target_base}/images/train", exist_ok=True)
os.makedirs(f"{target_base}/labels/train", exist_ok=True)
os.makedirs(f"{target_base}/images/val", exist_ok=True)
os.makedirs(f"{target_base}/labels/val", exist_ok=True)

MAX_PER_CLASS = 100
# 클래스 이름 정의
classes = {"airplane": 0, "car": 1, "cat": 2, "dog": 3, "flower": 4, "fruit": 5, "motorbike": 6, "person": 7}
train_count = 0
val_count = 0

for cls, cls_id in classes.items():
    src_folder = os.path.join(src_base, cls)
    imgs = [f for f in os.listdir(src_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    random.shuffle(imgs)
    imgs = imgs[:MAX_PER_CLASS]
    split_idx = int(len(imgs) * 0.8)
    train_imgs, val_imgs = imgs[:split_idx], imgs[split_idx:]

    for subset, subset_imgs in [("train", train_imgs), ("val", val_imgs)]:
        for img_name in subset_imgs:
            src_img = os.path.join(src_folder, img_name)
            dst_img = os.path.join(f"{target_base}/images/{subset}", img_name)
            dst_txt = os.path.join(f"{target_base}/labels/{subset}", os.path.splitext(img_name)[0] + ".txt")

            shutil.copy(src_img, dst_img)

            with open(dst_txt, "w") as f:
                f.write(f"{cls_id} 0.5 0.5 1.0 1.0\n") # 객체 좌표값
        if subset == "train":
            train_count += len(subset_imgs)
        else:
            val_count += len(subset_imgs)

print(f"데이터 생성 완료!")
print(f"Train 이미지: {train_count}장")
print(f"Val 이미지: {val_count}장")
print(f"총합: {train_count + val_count}장")

데이터 생성 완료!
Train 이미지: 640장
Val 이미지: 160장
총합: 800장


In [38]:
# import yaml
# import os

# abs_train = os.path.abspath('content/data/labels/train')
# abs_val = os.path.abspath('content/data/labels/val')
# with open("data.yaml",'r') as f:
#   #data=yaml.load(f,Loader=yaml.FullLoader)
#   data=yaml.full_load(f)
# data['train']=abs_train
# data['val']=abs_val
# data['nc']=8
# data['names']=['airplane','car','cat','dog','flower','fruit','motorbike','person']
# with open("data.yaml",'w') as f:
#   yaml.dump(data,f)

In [39]:
import yaml
import os

# Define the data dictionary
data = {
    'train': os.path.abspath('data/images/train'), # ('data/labels/train') -> ('data/images/train')
    'val': os.path.abspath('data/images/val'),     # ('data/labels/val') -> ('data/images/val')
    'nc': 8, # Corrected the number of classes to 8 based on the 'classes' dictionary
    'names': ['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']
}

# Write the data to data.yaml
with open("data.yaml", 'w') as f:
    yaml.dump(data, f)

# Verify the content of data.yaml
with open("data.yaml", 'r') as f:
    print(f.read())

names:
- airplane
- car
- cat
- dog
- flower
- fruit
- motorbike
- person
nc: 8
train: /content/data/images/train
val: /content/data/images/val



In [42]:
!python yolov5/train.py --img 256 --batch 32 --epoch 100 --data data.yaml --cfg yolov5/models/yolov5m.yaml

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-11-04 10:44:10.204888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762253050.226038   21761 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762253050.232471   21761 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762253050.248145   21761 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762253050.248170   21761 computation_placer.cc:177] computation placer already registere

In [57]:
model_path = "/content/yolov5/runs/train/exp4/weights/best.pt"
test_img_list = []
test_img_list.append("/content/data/images/val/airplane_0248.jpg")
test_img_list.append("/content/data/images/val/car_0195.jpg")
test_img_list.append("/content/data/images/val/cat_0459.jpg")
test_img_list.append("/content/data/images/val/dog_0420.jpg")
test_img_list.append("/content/data/images/val/flower_0173.jpg")
test_img_list.append("/content/data/images/val/fruit_0988.jpg")
test_img_list.append("/content/data/images/val/motorbike_0333.jpg")
test_img_list.append("/content/data/images/val/person_0382.jpg")

In [58]:
for test_img_path in test_img_list:
    !python yolov5/detect.py --weights "/content/yolov5/runs/train/exp4/weights/best.pt" --img 256 --source $test_img_path


detect: weights=['/content/yolov5/runs/train/exp4/weights/best.pt'], source=/content/data/images/val/airplane_0248.jpg, data=yolov5/data/coco128.yaml, imgsz=[256, 256], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-446-ga53364d6 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/data/images/val/airplane_0248.jpg: 96x256 1 airplane, 48.7ms
Speed: 0.3ms pre-process, 48.7ms inference, 110.5ms NMS per image at shape (1, 3, 256, 256)
Results saved to yolov5/runs/detect/exp19
detect: weights=['/content/yolov5/runs/tra